In [1]:
import torch
#from transformers import Blip2VisionConfig, Blip2VisionModel, AutoModelForImageClassification
import copy, os, sys
sys.path.append('../')
#from minigpt4.models.eva_vit import VisionTransformer, create_eva_vit_g
from minigpt4.models.mini_gpt4 import MiniGPT4


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /hetu_group/wangyan/envs/env_minigpt4/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113_nocublaslt.so...


/hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/hetu_group/wangyan/envs/env_minigpt4/lib/libcudart.so'), PosixPath('/hetu_group/wangyan/envs/env_minigpt4/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
/hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


/export/home/.cache/minigpt4


In [2]:
qformer, query_tokens = MiniGPT4.init_Qformer(32,1408)

In [3]:
tmp = qformer.config

In [4]:
qformer.config = tmp
qformer.config

BertConfig {
  "add_cross_attention": true,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cross_attention_freq": 2,
  "encoder_width": 1408,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "query_length": 32,
  "transformers_version": "4.31.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [12]:
from peft import LoraConfig, get_peft_config, get_peft_model

qformer.config = None
config = LoraConfig(
    adapter_name = 'det',
    r=16,
    lora_alpha=16,
    target_modules=['.query','.key'],
    lora_dropout=0.1,
    bias="none",
)
lora_model = get_peft_model(qformer, config)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 from peft import LoraConfig, get_peft_config, get_peft_model                                │
│    2                                                                                             │
│    3 qformer.config = None                                                                       │
│ ❱  4 config = LoraConfig(                                                                        │
│    5 │   adapter_name = 'det',                                                                   │
│    6 │   r=16,                                                                                   │
│    7 │   lora_alpha=16,                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: __init__() got an unexpected keyword argument 'adapter_name'

In [13]:
lora_model

PeftModel(
  (base_model): LoraModel(
    (model): BertLMHeadModel(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(
                    in_features=768, out_features=768, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
           

In [6]:
qformer

BertLMHeadModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inpla

In [ ]:
# from IPython.core.debugger import set_trace; set_trace()

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [9]:
print_trainable_parameters(lora_model)

trainable params: 946176 || all params: 187228218 || trainable%: 0.51


In [4]:
from peft import LoraConfig, get_peft_config, get_peft_model
s_mini_ve.config = None
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=['query','key'],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(s_mini_ve, config)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 from peft import LoraConfig, get_peft_config, get_peft_model                                │
│ ❱  2 s_mini_ve.config = None                                                                     │
│    3 config = LoraConfig(                                                                        │
│    4 │   r=16,                                                                                   │
│    5 │   lora_alpha=16,                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 's_mini_ve' is not defined

In [ ]:
lora_model

In [37]:
lora_model.base_model.blocks[0].attn.query.lora_B.default.weight.shape

torch.Size([1408, 16])

In [31]:
mini_ve.eval()
lora_model.eval()
input_image = torch.randn([1,3,224,224])
out_ns = mini_ve(input_image)
out_s = s_mini_ve(input_image)

In [32]:
torch.max(out_ns - out_s)

tensor(0., grad_fn=<MaxBackward1>)

In [33]:
False in (out_ns == out_s)

False

In [38]:
out_ns.shape

torch.Size([1, 257, 1408])

In [39]:
257*1408

361856

In [34]:
mini_ve

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (a

In [35]:
vision_config_hv = {'hidden_dropout_prob':0.0}

In [36]:
from transformers import ViTModel

In [11]:
hvit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

In [ ]:
vision_config = Blip2VisionConfig()

In [6]:
blip_ve = Blip2VisionModel(vision_config)

In [9]:
or_bve = copy.deepcopy(blip_ve)

In [25]:
%%bash
export http_proxy=http://oversea-squid4.sgp.txyun:11080 https_proxy=http://oversea-squid4.sgp.txyun:11080 no_proxy=localhost,127.0.0.1,localaddress,localdomain.com,internal,corp.kuaishou.com,test.gifshow.com,staging.kuaishou.com

In [28]:
automodel = AutoModelForImageClassification.from_pretrained(
    pretrained_model_name_or_path = "google/vit-base-patch16-224-in21k",  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/transformers/utils/hub.py:417  │
│ in cached_file                                                                                   │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/huggingface_hub/utils/_validat │
│ ors.py:118 in _inner_fn                                                                          │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │                                                                                      │
│ ❱ 118 │   │   return fn(*args, **kwargs)                                                         │
│   119 │                                                                                          │
│   120 │   return _inner_fn  # type: ignore                                                       │
│   121                                                                                            │
│                                                                                                  │
│ /hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/huggingface_hub/file_download. │
│ py:1291 in hf_hub_download                                                                       │
│                                                                                                  │
│   1288 │   │   │   │   " and downloads online, set 'local_files_only' to False."                 │
│   1289 │   │   │   )                                                                             │
│   1290 │   │   else:                                                                             │
│ ❱ 1291 │   │   │   raise LocalEntryNotFoundError(                                                │
│   1292 │   │   │   │   "Connection error, and we cannot find the requested files in"             │
│   1293 │   │   │   │   " the disk cache. Please try again or make sure your Internet"            │
│   1294 │   │   │   │   " connection is on."                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
LocalEntryNotFoundError: Connection error, and we cannot find the requested files in the disk cache. Please try 
again or make sure your Internet connection is on.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 automodel = AutoModelFo

In [26]:
vision_config

Blip2VisionConfig {
  "attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "gelu",
  "hidden_size": 1408,
  "image_size": 224,
  "initializer_factor": 1.0,
  "initializer_range": 1e-10,
  "intermediate_size": 6144,
  "layer_norm_eps": 1e-05,
  "model_type": "blip_2_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 39,
  "patch_size": 14,
  "projection_dim": 512,
  "qkv_bias": true,
  "transformers_version": "4.31.0.dev0"
}